# Import Packages

### Copyright June Skeeter
Attribution 4.0 International (CC BY 4.0) 

Source:
https://github.com/June-Skeeter/GPX_Code

In [1]:

! pip install gpxpy
! pip install ipywidgets

import os
import gpxpy
import gpxpy.gpx
from zipfile import ZipFile
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString
import matplotlib.pyplot as plt
from datetime import datetime
from ipywidgets import FloatProgress, HTML
from IPython.display import display

def GPX_to_GeoDataFrame(gpx_file,CRS='EPSG:4326',verbose='False'):
    
    T1 = datetime.now()
    Point_Layer = {
    'Name':[],
    'Type':[],
    'Lat':[],
    'Lon':[],
    'Z':[],
    'Time':[],
    'HR':[]
    }
    gpx_file = open(gpx_file,'r')
    gpx = gpxpy.parse(gpx_file)
    for track in gpx.tracks:
        Name = track.name
        Type = track.type
        for segment in track.segments:
            for point in segment.points:
                Point_Layer['Lat'].append(point.latitude)
                Point_Layer['Lon'].append(point.longitude)
                Point_Layer['Z'].append(point.elevation)
                Point_Layer['Time'].append(point.time)
                Point_Layer['Name'].append(Name)
                Point_Layer['Type'].append(Type)
                hrx = 0
                for ext in point.extensions:
                    for extchild in list(ext):
                        if extchild.tag[-2:] == 'hr':
                            Point_Layer['HR'].append(extchild.text)
                            hrx = 1
                if hrx == 0:
                    Point_Layer['HR'].append(np.nan)
                    
           
    if verbose == 'True':
        print()
        for p in Point_Layer.keys():
            print(p)
            print(len(Point_Layer[p]))
        
    df = pd.DataFrame(Point_Layer)
        
    Points = gpd.GeoDataFrame(df,
                           geometry=gpd.points_from_xy(df['Lon'],df['Lat']),
                           crs='EPSG:4326'
                           )
    Points = Points.to_crs(CRS)
    
    Points['Time'] = pd.DatetimeIndex(Points['Time'])
    Points['Seconds']=Points['Time'].diff().dt.seconds
    Points['Delta_Z']=Points['Z'].diff()
    Points2 = Points.shift()
    Points['Key']=Points.index
    Points2['Key']=Points2.index
    Points_to_Lines=Points.append(Points2)
    Points_to_Lines=Points_to_Lines.sort_values(by='Key')
    Temp = Points_to_Lines[2:].groupby('Key').geometry.apply(lambda x: LineString(x.tolist()))
    Lines = Points_to_Lines[2:].copy()
    Lines.geometry=Temp
    Lines['Distance']=Lines.length
    Lines['Speed']=Lines['Distance']/Lines['Seconds']
    Lines['Time'] = Lines['Time'].astype(str)
    Lines['HR'] = Lines['HR'].astype(float)
#     Lines=Lines.drop(['Lat','Lon'],axis=1)
     
    return(Lines)

# Read and Save Files

In [2]:
Batch_Name = 'Test_Batch'

# Zip = []
T1 = datetime.now()

current = HTML(
    value="",    placeholder='File: ',    description='File: ',
)
display(current)



i = 0
prog = FloatProgress(min=0, max=100,description='Progress:')
prog.value=0
display(prog)

Fail = []

for path,dirs,files in os.walk('Inputs/'):
    for f in files:
        try:
            current.value=' '+ f
            gpx_file = path+f
            DataFrame=GPX_to_GeoDataFrame(gpx_file,CRS='EPSG:3005')
            DataFrame.to_file('Outputs/'+f.split('.')[0]+'.shp')
            DataFrame.drop('geometry',axis=1).to_csv('Outputs/'+f.split('.')[0]+'.csv')
            i += 1
            prog.value=i/len(files)*100
        except:
            Fail.append(f)
            pass

zipObj = ZipFile(Batch_Name + '.zip', 'w')


for path,dirs,files in os.walk('Outputs/'):
    for f in files:
        if f != 'Keep.txt':
            current.value=f
            zipObj.write(path+f)
            os.remove(path+f)
zipObj.close()

print('Done Time Elapsed',datetime.now()-T1)
print('Failures: ',Fail)

HTML(value='', description='File: ', placeholder='File: ')

FloatProgress(value=0.0, description='Progress:')

Done Time Elapsed 0:00:18.091192
Failures:  []
